1.4.4. Выбираем ход в морском бое

Тип работы:  Индивидуальная работа

Мы расставили свои корабли - три однопалубных и один двухпалубный на поле 4х4. То же самое сделал и противник.

Мы сделали уже несколько ходов и даже потопили три однопалубных корабля противника. Мы получили запрос на новый ход.

Выбор нашего хода осуществляется с помощью функции

```python
def choose_act(battle_state, last_move, result):
  ...
  move = (letter, number)
  return move
```

где 

**last_move** - последний сделанный вами ход в виде кортежа `(letter, number)`, **letter** - это одна из букв `a,b,c,d`, а **number** - это целое число от 1 до 4. (см. 3.15. Расстановка кораблей для морского боя);

**result** - результат последнего хода в виде строки `'miss' | 'injury' | 'killed'` (промах, ранен, убит) - см. 3.16. Определяем результат хода.

**battle_state** - это двухмерный массив numpy 4х4, который содержит результат наших выстрелов в соответствии с правилами:

Пусть

`battle_state[i,j] == n`

тогда

если `n = 0`, то мы еще не стреляли по полю (i,j)
если `n < 0`, то мы стреляли по полю (i,j) и промахнулись
если `n == 1`, то мы стреляли по полю (i,j) и ранили корабль противника
если `n == 2`, то мы стреляли по полю (i,j) и корабль противника убит

**ПРИМЕР 1**. Пусть мы имеем такое поле боя после наших пяти выстрелов

battle-1

Т.е. мы подбили уже 3 одиночных корабля и два раза промахнулись в (a,1) и (b,1).

Тогда на вход нашей функции мы получим такой numpy массив **battle_state**:

`numpy.array([[-1,-1,2,0],[0,0,0,0],[0,2,0,0],[2,0,0,0]])`

Ваша задача - написать функцию `choose_act(battle_state)`

так, чтобы она выбирала один из возможных рациональных ходов - поле, где может находиться оставшийся корабль противника.

Для данного примера ваша функция должна возвратить один, любой из оставшихся рациональных ходов `(d,2), (d,3), (d,4), (c,4)`

(см. правила расстановки кораблей в упражнении 3.15)

In [23]:
import numpy as np

def is_2_deck_ship_dead(battle_state):
    for i in range(battle_state.shape[0]):
        for j in range(battle_state.shape[1]):
            if battle_state[i, j] == 2:
                if i > 0:   # up
                    if battle_state[i - 1, j] == 2:
                        return True
                if j < 3:   # right
                    if battle_state[i, j + 1] == 2:
                        return True
                if i < 3:   # down
                    if battle_state[i + 1, j] == 2:
                        return True
                if j > 0:   # left
                    if battle_state[i, j - 1] == 2:
                        return True
    return False

def kill_boat(battle_state, last_move):
    '''
    :param battle_state: двухмерный массив numpy 4х4, который содержит результат наших выстрелов
    battle_state[i,j] < 0, то стреляли по полю (i,j) и ничего, 
    battle_state[i,j] == 0, то не стреляли; если 1 или 2 - ранили и убили
    :param last_move: последний ваш ход (letter, number),который ранил 2-х палубный 
    :return: move=(letter, number) - выбранный ход, чтобы добить корабль
    '''
    move = ("z", 0) # пустой ход

    cti = str.maketrans({'a': '1', 'b': '2', 'c': '3', 'd': '4' })
    last_move_i = (int(last_move[0].translate(cti)), last_move[1])
    itc = "abcd"

    print("last_move_i:", last_move_i)

    move_candidates = []
    # up
    if (last_move_i[1] - 1) > 0:
        move_candidates.append(tuple((last_move_i[0], last_move_i[1] - 1)))
    # right
    if (last_move_i[0] + 1) < 5:
        move_candidates.append(tuple((last_move_i[0] + 1, last_move_i[1])))
    # down
    if (last_move_i[1] + 1) < 5:
        move_candidates.append(tuple((last_move_i[0], last_move_i[1] + 1)))
    # left
    if (last_move_i[0] - 1) > 0:
        move_candidates.append(tuple((last_move_i[0] - 1, last_move_i[1])))
    print(move_candidates)

    for mc in move_candidates:
        # clean coordinates: [1-4] -> [0-3]
        # also reversed, because map coordinates (move, last_move) are in the form [column, row]
        # and battle_state is in the form [row, column]
        coords = (mc[1]-1, mc[0]-1)

        if battle_state[coords[0], coords[1]] != 0:
            continue
        if (coords[0]-1) >= 0:
            if battle_state[coords[0]-1, coords[1]] == 2:
                continue
        if (coords[1]+1) <= 3:
            if battle_state[coords[0], coords[1]+1] == 2:
                continue
        if (coords[0]+1) <= 3:
            if battle_state[coords[0]+1, coords[1]] == 2:
                continue
        if (coords[1]-1) >= 0:
            if battle_state[coords[0], coords[1]-1] == 2:
                continue
        move = (itc[coords[1]], mc[1])
        break

    return move


def check_neighbors(move, battle_state):
    if move[0] > 0: # up
        if battle_state[move[0] - 1, move[1]] == 2:
            return False
    if move[1] < 3: # right
        if battle_state[move[0], move[1] + 1] == 2:
            return False
    if move[0] < 3: # down
        if battle_state[move[0] + 1, move[1]] == 2:
            return False
    if move[1] > 0: # left
        if battle_state[move[0], move[1] - 1] == 2:
            return False
    return True


# ---------- функция выбора нового хода ----------
def choose_act(battle_state, last_move, result):
    # battle_state - это двухмерный массив numpy 4х4, который содержит результат наших выстрелов
    # last_move - последний сделанный вами ход в виде кортежа (letter, number),
    # letter - это одна из букв a,b,c,d, а number - это целое число от 1 до 4. (см. 3.15. Расстановка кораблей для морского боя);
    # result - результат последнего хода в виде строки 'miss' | 'injury' | 'killed' (промах, ранен, убит) - см. 3.16. Определяем результат хода.

    move = ("z", 0)
    new_move_is_determined = False

    # finish off 2-deck ship
    for i in range(4):
        for j in range(4):
            if battle_state[i, j] == 1:
                move = kill_boat(battle_state, ('abcd'[j], i + 1))
                return move

    two_deck_ship_is_dead = is_2_deck_ship_dead(battle_state)

    for i in range(4):
        if new_move_is_determined:
            break
        for j in range(4):
            if battle_state[i, j] != 0:
                continue
            # check self
            if not check_neighbors((i, j), battle_state):
                continue
            if two_deck_ship_is_dead:   # search for 1-deck ship
                move = ('abcd'[j], i + 1)
                return move
            else:                       # search for 2-deck ship
                # check possible second decks
                if i > 0:   # up
                    if battle_state[i - 1, j] == 0 and check_neighbors((i - 1, j), battle_state):
                        move = ('abcd'[j], i + 1)
                        new_move_is_determined = True
                        break
                if j < 3:   # right
                    if battle_state[i, j + 1] == 0 and check_neighbors((i, j + 1), battle_state):
                        move = ('abcd'[j], i + 1)
                        new_move_is_determined = True
                        break
                if i < 3:   # down
                    if battle_state[i + 1, j] == 0 and check_neighbors((i + 1, j), battle_state):
                        move = ('abcd'[j], i + 1)
                        new_move_is_determined = True
                        break
                if j > 0:   # left
                    if battle_state[i, j - 1] == 0 and check_neighbors((i, j - 1), battle_state):
                        move = ('abcd'[j], i + 1)
                        new_move_is_determined = True
                        break
    return move

# battle_state = np.array([[-1,-1,2,0],[0,0,0,0],[0,2,0,0],[2,0,0,0]])
# last_move = ('c', 1)
# result = 'killed'
# ('d', 2)

# battle_state = np.array([[0,-1,0,0], [0,0,2,0], [2,0,0,0], [0,1,0,-1]])
# last_move = ('d', 4)
# result = 'miss'
# ('c', 4)

battle_state = np.array([[0,0,2,-1], [2,0,2,0], [0,-1,0,0], [-1,0,0,2]])
last_move = ('c', 2)
result = 'kill'

choose_act(battle_state, last_move, result)

('b', 4)

In [ ]:
battle_state = np.array([[0,-1,0,0], [0,0,2,0], [2,0,0,0], [0,1,0,-1]])
